In [1]:
from pyspark.sql import *

spark = SparkSession \
    .Builder() \
    .master("local[*]") \
    .appName("Filter_Total_Money/Category(RDD)") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/22 23:19:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = spark.read.csv("/Users/dangvu/My_Project/Data/products.csv", header=True, inferSchema=True)

In [3]:
df.take(2)

[Row(ID='EM9579', Tên sản phẩm='Đầm Suông Đen Phi cc', Giá='750.000\xa0₫', Liên kết='https://vayemy.vn/chi-tiet/em9579-dam-suong-den-phi-cc', Hình ảnh='https://vayemy.vn/wp-content/uploads/2025/02/z6326384063968_414762ecbb0937ecf29c1e29a3f73f31-247x371.webp'),
 Row(ID='EM2113', Tên sản phẩm='Đầm Xoè Đỏ Kết Đá', Giá='780.000\xa0₫', Liên kết='https://vayemy.vn/chi-tiet/em2113-dam-xoe-do-ket-da', Hình ảnh='https://vayemy.vn/wp-content/uploads/2025/02/z6283389324224_11b12d1d2239a2b27442d147e1d9c005-247x371.webp')]

In [4]:
from pyspark.sql.functions import *
# Tách "Danh mục sản phẩm" từ cột "Tên sản phẩm"
# => lấy từ đầu tiên (index 0) của chuỗi
df_with_category = df.withColumn(
    "Danh mục",
    split(col("Tên sản phẩm"), " ").getItem(0)
)

In [5]:
df_with_category.show()

+------+--------------------+---------+--------------------+--------------------+--------+
|    ID|        Tên sản phẩm|      Giá|            Liên kết|            Hình ảnh|Danh mục|
+------+--------------------+---------+--------------------+--------------------+--------+
|EM9579|Đầm Suông Đen Phi cc|750.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|
|EM2113|   Đầm Xoè Đỏ Kết Đá|780.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|
|EM7669| Đầm Xoè Trắng Linen|780.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|
|EM9542|Đầm Xoè Phối Hoa ...|640.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|
|EM9565|Đầm dự tiệc suông...|720.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|
|EM2096|Đầm Tuyết Cotton ...|750.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|
|EM9577|   Đầm Xoè A Gấm Hoa|680.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|
|EM2101|Đầm xoè Gấm Hoa Vàng|750.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|

In [34]:
# Làm sạch giá: bỏ ký tự ₫, dấu . và khoảng trắng đặc biệt, rồi ép sang double
df_with_price = df_with_category.withColumn(
    "money",
    regexp_replace(col("Giá"), "[₫\\.\\u00A0]", "") \
    .cast("double")
)


In [20]:
df_with_price.show()

+------+--------------------+---------+--------------------+--------------------+--------+--------+
|    ID|        Tên sản phẩm|      Giá|            Liên kết|            Hình ảnh|Danh mục|   money|
+------+--------------------+---------+--------------------+--------------------+--------+--------+
|EM9579|Đầm Suông Đen Phi cc|750.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|750000.0|
|EM2113|   Đầm Xoè Đỏ Kết Đá|780.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|780000.0|
|EM7669| Đầm Xoè Trắng Linen|780.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|780000.0|
|EM9542|Đầm Xoè Phối Hoa ...|640.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|640000.0|
|EM9565|Đầm dự tiệc suông...|720.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|720000.0|
|EM2096|Đầm Tuyết Cotton ...|750.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|750000.0|
|EM9577|   Đầm Xoè A Gấm Hoa|680.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|680000.0|


In [35]:
# Giữ cặp (Danh mục sản phẩm, Giá)
df_with_price.select("Danh mục", "money")

DataFrame[Danh mục: string, money: double]

In [36]:
df_with_price.show()

+------+--------------------+---------+--------------------+--------------------+--------+--------+
|    ID|        Tên sản phẩm|      Giá|            Liên kết|            Hình ảnh|Danh mục|   money|
+------+--------------------+---------+--------------------+--------------------+--------+--------+
|EM9579|Đầm Suông Đen Phi cc|750.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|750000.0|
|EM2113|   Đầm Xoè Đỏ Kết Đá|780.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|780000.0|
|EM7669| Đầm Xoè Trắng Linen|780.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|780000.0|
|EM9542|Đầm Xoè Phối Hoa ...|640.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|640000.0|
|EM9565|Đầm dự tiệc suông...|720.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|720000.0|
|EM2096|Đầm Tuyết Cotton ...|750.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|750000.0|
|EM9577|   Đầm Xoè A Gấm Hoa|680.000 ₫|https://vayemy.vn...|https://vayemy.vn...|     Đầm|680000.0|


In [37]:
# Gom nhóm và tính tổng tiền
df_total = df_with_price.groupBy("Danh mục") \
    .agg(sum("money").alias("total_money"))


In [38]:
df_total.show()

+--------+-----------+
|Danh mục|total_money|
+--------+-----------+
|      Sự|   680000.0|
|     Sét|  1430000.0|
|    body|   640000.0|
|     Nét|   680000.0|
|      Áo|  1230000.0|
|     Sắc|   680000.0|
|     Jum|   750000.0|
|    Chân|   900000.0|
|     Đầm|    3.448E7|
|    Sườn|   720000.0|
|     Xoè|  1360000.0|
+--------+-----------+



In [ ]:
# Lọc theo total_money > 2 triệu
df_filtered = df_total.filter(col("total_money") > 20000000)

df_filtered.show(10, truncate=False)

+--------+-----------+
|Danh mục|total_money|
+--------+-----------+
|Đầm     |3.448E7    |
+--------+-----------+



In [40]:
spark.stop()